In [22]:
#Creating new tables better resolution
#No access to data so tables are created manually

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pandas.plotting import table
import imgkit

'''
    
19-20 1st part	19-20 2nd part	20-21
Home	434	142	139
Away	350	108	151

'''

df_scored = pd.DataFrame({ '19-20 1st part': [434, 350], \
'19-20 2nd part': [142, 108], '20-21': [139, 151]}, \
index=['Home', 'Away'])

df_scored = df_scored.style.apply(lambda x: ['Background-color: #09bb9f' if x[i]== x.max() else \
'Background-color: #c4c4c4' for i in range(len(x))])

df_scored = df_scored.set_table_styles([{'selector': 'td', 'props': [('border', 'solid 1px')]},\
{'selector': 'th', 'props': [('border', 'solid 1px')]}])

html_scored = df_scored.render()

# ax = plt.subplot(111, frame_on=False) # no visible frame
# ax.xaxis.set_visible(False)  # hide the x axis
# ax.yaxis.set_visible(False)  # hide the y axis

# table(ax, df_scored)  # where df is your data frame

# plt.savefig('mytable.png', dpi=500)

html_scored = df_scored.render()

df_scored

,19-20 1st part,19-20 2nd part,20-21
Home,434,142,139
Away,350,108,151


In [43]:
# with open(Path('test.html'), 'w') as file:
#     file.write(html_scored)
# path_wkthmltoimage = r'C:\Program Files\wkhtmltopdf\bin\wkhtmltoimage.exe'
# config = imgkit.config(wkhtmltoimage=path_wkthmltoimage)
# options = {
#     'zoom': 4,
#     'quality': 100
# }
# imgkit.from_file('scored.html', 'scored.jpg', config=config, options=options)

tbls = ['scored', 'xG', 'reg', 'playersH', 'playersA']
path_wkthmltoimage = r'C:\Program Files\wkhtmltopdf\bin\wkhtmltoimage.exe'
config = imgkit.config(wkhtmltoimage=path_wkthmltoimage)
options = {
    'zoom': 2,
    'quality': 100
}
for i in tbls:
    imgkit.from_file(f'{i}.html', f'{i}.jpg', config=config, options=options)

df_scored

Loading page (1/2)
Rendering (2/2)                                                    
Done                                                               
Loading page (1/2)
Rendering (2/2)                                                    
Done                                                               
Loading page (1/2)
Rendering (2/2)                                                    
Done                                                               
Loading page (1/2)
Rendering (2/2)                                                    
Done                                                               
Loading page (1/2)
Rendering (2/2)                                                    
Done                                                               


,19-20 1st part,19-20 2nd part,20-21
Home,434,142,139
Away,350,108,151


In [1]:
#%pylab inline
import sklearn
from sklearn.linear_model import LinearRegression
import numpy as np
import pandas as pd
from pathlib import Path
from Brr_functions import no_lists
import json
import imgkit

def x_maker( NAdf,  hist_num, NAdf_answers, NAdf_opp, NAdf_home, threatAllowedAv):
    '''
        Making table X for regression where last column ia an aswer.
        Adjusted tables used for lag data. UnAdjusted for answer and opponent strength.
    '''
    #Removes empty fixtures
    df = NAlast(NAdf)
    df_answers =NAlast(NAdf_answers)
    df_opp = NAlast(NAdf_opp)
    df_home = NAlast(NAdf_home)
    
    X = pd.DataFrame()
    col = []

    for i in range(hist_num, 0, -1): #Creating column names for lag columns
        col.append(f'GW-{i}')

    for i in range(0,len(df.columns)-hist_num):
        #print(i+1)
        next_tour2add = df[df.columns[i:i+hist_num]].copy()#no_lists(df.sort_index())[df.columns[i:i+6]]
        next_tour2add.columns=col#['GW-6','GW-5','GW-4','GW-3','GW-2','GW-1']
        next_tour2add['Home'] = df_home[df_home.columns[hist_num+i]]
        next_tour2add['Opponent'] = (df_opp[df_opp.columns[hist_num+i]]*threatAllowedAv)#.copy()
        next_tour2add['GW'] = df_answers[df_answers.columns[hist_num+i]]#.copy()#.tolist()
        X = X.append(next_tour2add, ignore_index=True, sort=False)
    print(len(X.dropna()))
    return X.dropna()

def NAlast(df):
    '''
        Removes NaN holes in the table
    '''
    def NAlast_row(row):
        '''Removes holes in a row. Subfunction of NAlast.'''
        b = list(row.dropna())
        for i in range(len(b), len(row)):
            b.append(np.nan)
        return b
    dff = df.copy()
    for i in df.index:
        dff.loc[i] = NAlast_row(dff.loc[i])
    return dff

'''
    DOWNOADING TABLES FROM MID AND
    Making all input tables standart format. Only numBer columns. From left to right. No lists, columns for DGW added
'''

'''
    import curent year data
'''
with open(Path('mid/FPL/Team_xG.txt'), 'r') as file:
    Team_xG = pd.DataFrame(json.loads(file.read()))
    Team_xG.index = pd.to_numeric(Team_xG.index)
    TxG = no_lists(Team_xG.sort_index())[no_lists(Team_xG).columns[:3:-1]]
    
with open(Path('mid/FPL/Team_xG_Ad.txt'), 'r') as file:
    Team_xG_Ad = pd.DataFrame(json.loads(file.read()))
    Team_xG_Ad.index = pd.to_numeric(Team_xG_Ad.index)
    TxGA = no_lists(Team_xG_Ad.sort_index())[TxG.columns]
    #display(Team_xG_Ad)
    
with open(Path('mid/FPL/Team_Opponent_xG.txt'), 'r') as file:
    Team_Opponent_xG = pd.DataFrame(json.loads(file.read()))
    Team_Opponent_xG.index = pd.to_numeric(Team_Opponent_xG.index)
    TOxG = no_lists(Team_Opponent_xG.sort_index())[TxG.columns]
    
with open(Path('mid/FPL/Team_oppdefence_coeff.txt'), 'r') as file:
    Team_oppdefence_coeff = pd.DataFrame(json.loads(file.read()))
    Team_oppdefence_coeff.index = pd.to_numeric(Team_oppdefence_coeff.index)
    Todc = no_lists(Team_oppdefence_coeff.sort_index())[TxG.columns]
    
Team_home = pd.read_csv(Path(f'in/Team_home.csv'))
Player_home = pd.read_csv(Path(f'in/Player_home.csv'))

'''
    import 2019-2020 data
'''
year = '2019-2020'
if year=='': folder = ''
else: folder = f'history/{year}/'
with open(Path(f'{folder}mid/FPL/Team_xG.txt'), 'r') as file:
    Team_xG = pd.DataFrame(json.loads(file.read()))
    Team_xG.index = pd.to_numeric(Team_xG.index)
    TxG1920 = no_lists(Team_xG.sort_index())[no_lists(Team_xG).columns[:3:-1]]
    
with open(Path(f'{folder}mid/FPL/Team_xG_Ad.txt'), 'r') as file:
    Team_xG_Ad = pd.DataFrame(json.loads(file.read()))
    Team_xG_Ad.index = pd.to_numeric(Team_xG_Ad.index)
    TxGA1920 = no_lists(Team_xG_Ad.sort_index())[TxG1920.columns]
    #display(Team_xG_Ad)
    
with open(Path(f'{folder}mid/FPL/Team_Opponent_xG.txt'), 'r') as file:
    Team_Opponent_xG = pd.DataFrame(json.loads(file.read()))
    Team_Opponent_xG.index = pd.to_numeric(Team_Opponent_xG.index)
    TOxG1920 = no_lists(Team_Opponent_xG.sort_index())[TxG1920.columns]
    
with open(Path(f'{folder}mid/FPL/Team_oppdefence_coeff.txt'), 'r') as file:
    Team_oppdefence_coeff = pd.DataFrame(json.loads(file.read()))
    Team_oppdefence_coeff.index = pd.to_numeric(Team_oppdefence_coeff.index)
    Todc1920 = no_lists(Team_oppdefence_coeff.sort_index())[TxG1920.columns]
    
Team_home1920 = pd.read_csv(Path(f'{folder}in/Team_home.csv'))
Player_home1920 = pd.read_csv(Path(f'{folder}in/Player_home.csv'))

#Understat Data

'''
    import curent year data
'''
with open(Path('mid/Understat/Team_xG.txt'), 'r') as file:
    UnTeam_xG = pd.DataFrame(json.loads(file.read()))
    UnTeam_xG.index = pd.to_numeric(UnTeam_xG.index)
    UTxG = no_lists(UnTeam_xG.sort_index())[no_lists(UnTeam_xG).columns[:3:-1]]
    
with open(Path('mid/Understat/Team_xG_Ad.txt'), 'r') as file:
    UnTeam_xG_Ad = pd.DataFrame(json.loads(file.read()))
    UnTeam_xG_Ad.index = pd.to_numeric(UnTeam_xG_Ad.index)
    UTxGA = no_lists(UnTeam_xG_Ad.sort_index())[UTxG.columns]
    
with open(Path('mid/Understat/Team_Opponent_xG.txt'), 'r') as file:
    UnTeam_Opponent_xG = pd.DataFrame(json.loads(file.read()))
    UnTeam_Opponent_xG.index = pd.to_numeric(UnTeam_Opponent_xG.index)
    UTOxG = no_lists(UnTeam_Opponent_xG.sort_index())[TxG.columns]
    
with open(Path('mid/Understat/Team_oppdefence_coeff.txt'), 'r') as file:
    UnTeam_oppdefence_coeff = pd.DataFrame(json.loads(file.read()))
    UnTeam_oppdefence_coeff.index = pd.to_numeric(UnTeam_oppdefence_coeff.index)
    UTodc = no_lists(UnTeam_oppdefence_coeff.sort_index())[UTxG.columns]
    
'''
    import 2019-2020 data
'''
year = '2019-2020'
if year=='': folder = ''
else: folder = f'history/{year}/'
with open(Path(f'{folder}mid/Understat/Team_xG.txt'), 'r') as file:
    UnTeam_xG = pd.DataFrame(json.loads(file.read()))
    UnTeam_xG.index = pd.to_numeric(UnTeam_xG.index)
    UTxG1920 = no_lists(UnTeam_xG.sort_index())[no_lists(UnTeam_xG).columns[:3:-1]]
    
with open(Path(f'{folder}mid/Understat/Team_xG_Ad.txt'), 'r') as file:
    UnTeam_xG_Ad = pd.DataFrame(json.loads(file.read()))
    UnTeam_xG_Ad.index = pd.to_numeric(UnTeam_xG_Ad.index)
    UTxGA1920 = no_lists(UnTeam_xG_Ad.sort_index())[UTxG1920.columns]
    #display(Team_xG_Ad)
    
with open(Path(f'{folder}mid/Understat/Team_Opponent_xG.txt'), 'r') as file:
    UnTeam_Opponent_xG = pd.DataFrame(json.loads(file.read()))
    UnTeam_Opponent_xG.index = pd.to_numeric(UnTeam_Opponent_xG.index)
    UTOxG1920 = no_lists(UnTeam_Opponent_xG.sort_index())[UTxG1920.columns]
    
with open(Path(f'{folder}mid/Understat/Team_oppdefence_coeff.txt'), 'r') as file:
    UnTeam_oppdefence_coeff = pd.DataFrame(json.loads(file.read()))
    UnTeam_oppdefence_coeff.index = pd.to_numeric(UnTeam_oppdefence_coeff.index)
    UTodc1920 = no_lists(UnTeam_oppdefence_coeff.sort_index())[UTxG1920.columns]

display(UTOxG1920.mean(axis=1).mean())#That's threatAllowedAv actually


144.51298014769435

In [23]:
year = ''
if year=='': folder = ''
else: folder = f'history/{year}/'

Fixtures = pd.read_csv(Path(f'{folder}in/Fixtures.csv'))
Team_home = pd.read_csv(Path(f'{folder}in/Team_home.csv'))


Team_scored = pd.DataFrame()
for j in range(int(Fixtures['event'].max()),0,-1): 
    Team_scored['GW'+str(j)] = [Fixtures[((Fixtures['team_a']==i)|(Fixtures['team_h']==i))&(Fixtures['event']==j)]\
    [['team_h_score','team_a_score']].values for i in range(1, len(Team_xG)+1)]
Team_scored = Team_scored[Team_scored.columns[::-1]]

home_goals3 = \
(no_lists(Team_scored.applymap(lambda x: list(x)))\
.applymap(lambda x:np.nan if type(x)==float else x[0])*Team_home)\
.sum().sum()

away_goals3 = -(no_lists(Team_scored.applymap(lambda x: list(x)))\
.applymap(lambda x:np.nan if type(x)==float else x[1])*(Team_home-1))\
.sum().sum()

print(f'20-21\n Home: {home_goals3}\nAway: {away_goals3}\n\n')

year = '2019-2020'
if year=='': folder = ''
else: folder = f'history/{year}/'

Fixtures1920 = pd.read_csv(Path(f'{folder}in/Fixtures.csv'))
Team_home1920 = pd.read_csv(Path(f'{folder}in/Team_home.csv'))

Team_scored1920 = pd.DataFrame()
for j in range(int(Fixtures1920['event'].max()),0,-1):
    
    Team_scored1920['GW'+str(j)] = [Fixtures1920[((Fixtures1920['team_a']==i)|\
    (Fixtures1920['team_h']==i))&(Fixtures1920['event']==j)]\
    [['team_h_score','team_a_score']].values for i in range(1, len(Team_xG)+1)]
Team_scored1920 = Team_scored1920[Team_scored1920.columns[::-1]]


home_goals1 = \
(no_lists(Team_scored1920.applymap(lambda x: list(x)))\
.applymap(lambda x:np.nan if type(x)==float else x[0])*Team_home1920)[Team_home1920.columns[:30]]\
.sum().sum()

away_goals1 = -(no_lists(Team_scored1920.applymap(lambda x: list(x)))\
.applymap(lambda x:np.nan if type(x)==float else x[1])*(Team_home1920-1))[Team_home1920.columns[:30]]\
.sum().sum()

print(f'\n19-20\n\n Home1: {home_goals1}\nAway1: {away_goals1}')

home_goals2 = \
(no_lists(Team_scored1920.applymap(lambda x: list(x)))\
.applymap(lambda x:np.nan if type(x)==float else x[0])*Team_home1920)[Team_home1920.columns[30:]]\
.sum().sum()

away_goals2 = -(no_lists(Team_scored1920.applymap(lambda x: list(x)))\
.applymap(lambda x:np.nan if type(x)==float else x[1])*(Team_home1920-1))[Team_home1920.columns[30:]]\
.sum().sum()

print(f'\n19-20\n\n Home2: {home_goals2}\nAway2: {away_goals2}')

df_scored = pd.DataFrame({ '19-20 1st part': [int(home_goals1), int(away_goals1)], \
'19-20 2nd part': [int(home_goals2), int(away_goals2)], '20-21': [int(home_goals3), int(away_goals3)]}, \
index=['Home', 'Away'])

# df_scored.to_csv('text/01. H-A/Scored.csv')
# df_scored

df_scored = df_scored.style.apply(lambda x: ['Background-color: #09bb9f' if x[i]== x.max() else \
'Background-color: #c4c4c4' for i in range(len(x))])

df_scored = df_scored.set_table_styles([{'selector': 'td', 'props': [('border', 'solid 1px')]},\
{'selector': 'th', 'props': [('border', 'solid 1px')]}])
#('border-style','solid'),('border-width','0.2px')]}])

html_scored = df_scored.render()

with open(Path('text/01. H-A/scored.html'), 'w') as file:
    file.write(html_scored)
path_wkthmltoimage = r'C:\Program Files\wkhtmltopdf\bin\wkhtmltoimage.exe'
config = imgkit.config(wkhtmltoimage=path_wkthmltoimage)
imgkit.from_file('text/01. H-A/scored.html', 'text/01. H-A/scored.jpg', config=config)
df_scored

20-21
 Home: 139.0
Away: 151.0



19-20

 Home1: 434.0
Away1: 350.0

19-20

 Home2: 142.0
Away2: 108.0
Loading page (1/2)
Rendering (2/2)                                                    
Done                                                               


,19-20 1st part,19-20 2nd part,20-21
Home,434,142,139
Away,350,108,151


In [24]:
# X3 = x_maker(TxGA1920,  19, TxG1920, Todc1920, Team_home1920, TOxG1920.mean(axis=1).mean())
# X = pd.DataFrame(X3[X3.columns[:-3]].mean(axis=1))
# X['Home'] = X3['Home']
# X['Opponent'] = X3['Opponent']
# Y = X3['GW']
# reg = LinearRegression().fit(X, Y)
# display(reg.score(X, Y))
# display(reg.coef_)

print('FPL\n')

print('2020-2021 season\n')
home_xG_FPL3 = (TxG*Team_home).sum().sum()/98
away_xG_FPL3 = -(TxG*(Team_home-1)).sum().sum()/98
print(home_xG_FPL3)
print(away_xG_FPL3)
print('\n')

print('All 2019-2020 season\n')
print((TxG1920*Team_home1920).sum().sum()/380)
print(-(TxG1920*(Team_home1920-1)).sum().sum()/380)
print('\n')

print('1st part\n')
home_xG_FPL1 = (TxG1920[UTxG1920.columns[:30]]*Team_home1920[UTxG1920.columns[:30]]).sum().sum()/288
away_xG_FPL1 = -(TxG1920[UTxG1920.columns[:30]]*(Team_home1920[UTxG1920.columns[:30]]-1)).sum().sum()/288
print(home_xG_FPL1)
print(away_xG_FPL1)
print('\n')

print('2nd part\n')
home_xG_FPL2 = (TxG1920[UTxG1920.columns[30:]]*Team_home1920[UTxG1920.columns[30:]]).sum().sum()/92
away_xG_FPL2 = -(TxG1920[UTxG1920.columns[30:]]*(Team_home1920[UTxG1920.columns[30:]]-1)).sum().sum()/92
print(home_xG_FPL2)
print(away_xG_FPL2)

print('\n\nUnderstat\n')

print('2020-2021 season\n')
home_xG_Understat3 = (UTxG*Team_home).sum().sum()/98
away_xG_Understat3 = -(UTxG*(Team_home-1)).sum().sum()/98
print(home_xG_Understat3)
print(away_xG_Understat3)
print('\n')

print('All 2019-2020 season\n')
print((UTxG1920*Team_home1920).sum().sum()/380)
print(-(UTxG1920*(Team_home1920-1)).sum().sum()/380)
print('\n')

print('1st part\n')
home_xG_Understat1 = (UTxG1920[UTxG1920.columns[:30]]*Team_home1920[UTxG1920.columns[:30]]).sum().sum()/288
away_xG_Understat1 = -(UTxG1920[UTxG1920.columns[:30]]*(Team_home1920[UTxG1920.columns[:30]]-1)).sum().sum()/288
print(home_xG_Understat1)
print(away_xG_Understat1)
print('\n')

print('2nd part\n')
home_xG_Understat2 = (UTxG1920[UTxG1920.columns[30:]]*Team_home1920[UTxG1920.columns[30:]]).sum().sum()/92
away_xG_Understat2 = -(UTxG1920[UTxG1920.columns[30:]]*(Team_home1920[UTxG1920.columns[30:]]-1)).sum().sum()/92
print(home_xG_Understat2)
print(away_xG_Understat2)


df_xG = pd.DataFrame({ ' FPL 19-20 1st part': [int(home_xG_FPL1), int(away_xG_FPL1)], \
' Understat 19-20 1st part': [int(home_xG_Understat1), int(away_xG_Understat1)], \
' FPL 19-20 2nd part': [int(home_xG_FPL2), int(away_xG_FPL2)], \
' Understat 19-20 2nd part': [int(home_xG_Understat2), int(away_xG_Understat2)], \
'FPL 20-21': [int(home_xG_FPL3), int(away_xG_FPL3)], \
'Understat 20-21': [int(home_xG_Understat3), int(away_xG_Understat3)]}, \
index=['Home', 'Away'])

df_xG = df_xG.style.apply(lambda x: ['Background-color: #09bb9f' if x[i]== x.max() else \
'Background-color: #c4c4c4' for i in range(len(x))])

df_xG = df_xG.set_table_styles([{'selector': 'td', 'props': [('border', 'solid 1px')]},\
{'selector': 'th', 'props': [('border', 'solid 1px')]}])

html_xG = df_xG.render()



with open(Path('text/01. H-A/xG.html'), 'w') as file:
    file.write(html_xG)
path_wkhtmltoimage = r'C:\Program Files\wkhtmltopdf\bin\wkhtmltoimage.exe'
config = imgkit.config(wkhtmltoimage=path_wkthmltoimage)
imgkit.from_file('text/01. H-A/xG.html', 'text/01. H-A/xG.jpg', config=config)
df_xG

FPL

2020-2021 season

159.79591836734693
154.5


All 2019-2020 season

177.57368421052632
152.85


1st part

184.57986111111111
154.125


2nd part

155.6413043478261
148.8586956521739


Understat

2020-2021 season

142.16977576344897
139.73155291894594


All 2019-2020 season

155.1475886596179
133.8783716357708


1st part

160.08418822214858
135.9540163560358


2nd part

139.69388568126087
127.3807012071152
Loading page (1/2)
Rendering (2/2)                                                    
Done                                                               


,FPL 19-20 1st part,Understat 19-20 1st part,FPL 19-20 2nd part,Understat 19-20 2nd part,FPL 20-21,Understat 20-21
Home,184,160,155,139,159,142
Away,154,135,148,127,154,139


In [25]:
hist_num = 5

# df = TxGA1920
# df_answers = TxG1920
# df_opp = Todc1920
# df_home = Team_home1920

# threatAllowedAv = TOxG1920.mean(axis=1).mean()

print("FPl All 2019-2020 season\n")
df = x_maker(TxGA1920,  hist_num, TxG1920, Todc1920, Team_home1920, TOxG1920.mean(axis=1).mean())
X = pd.DataFrame(df[df.columns[:-3]].mean(axis=1))
X['Home'] = df['Home']
X['Opponent'] = df['Opponent']
Y = df['GW']
reg = LinearRegression().fit(X, Y)
display(reg.score(X, Y))
display(reg.coef_)
display(reg.intercept_)

print("Understat All 2019-2020 season\n")
df = x_maker(UTxGA1920,  hist_num, UTxG1920, UTodc1920, Team_home1920, UTOxG1920.mean(axis=1).mean())
X = pd.DataFrame(df[df.columns[:-3]].mean(axis=1))
X['Home'] = df['Home']
X['Opponent'] = df['Opponent']
Y = df['GW']
reg = LinearRegression().fit(X, Y)
display(reg.score(X, Y))
display(reg.coef_)
display(reg.intercept_)

print("\nFPL 1st part\n")
df = x_maker(TxGA1920[TxGA1920.columns[:30]],  hist_num, TxG1920[TxGA1920.columns[:30]],\
Todc1920[TxGA1920.columns[:30]], Team_home1920[TxGA1920.columns[:30]], TOxG1920[TxGA1920.columns[:30]].mean(axis=1).mean())
X = pd.DataFrame(df[df.columns[:-3]].mean(axis=1))
X['Home'] = df['Home']
X['Opponent'] = df['Opponent']
Y = df['GW']
reg = LinearRegression().fit(X, Y)
display(reg.score(X, Y))
display(reg.coef_)
display(reg.intercept_)
coeff_FPL1 = list(reg.coef_)

print("\nUnderstat 1st part\n")
df = x_maker(UTxGA1920[UTxGA1920.columns[:30]],  hist_num, UTxG1920[UTxGA1920.columns[:30]],\
UTodc1920[UTxGA1920.columns[:30]], Team_home1920[UTxGA1920.columns[:30]], UTOxG1920[UTxGA1920.columns[:30]].\
mean(axis=1).mean())
X = pd.DataFrame(df[df.columns[:-3]].mean(axis=1))
X['Home'] = df['Home']
X['Opponent'] = df['Opponent']
Y = df['GW']
reg = LinearRegression().fit(X, Y)
display(reg.score(X, Y))
display(reg.coef_)
display(reg.intercept_)
coeff_Understat1 = list(reg.coef_)

print("\nFPL 2nd part\n")
df = x_maker(TxGA1920[TxGA1920.columns[30:]],  hist_num, TxG1920[TxGA1920.columns[30:]],\
Todc1920[TxGA1920.columns[30:]], Team_home1920[TxGA1920.columns[30:]], TOxG1920[TxGA1920.columns[30:]].mean(axis=1).mean())
X = pd.DataFrame(df[df.columns[:-3]].mean(axis=1))
X['Home'] = df['Home']
X['Opponent'] = df['Opponent']
Y = df['GW']
reg = LinearRegression().fit(X, Y)
display(reg.score(X, Y))
display(reg.coef_)
display(reg.intercept_)
coeff_FPL2 = list(reg.coef_)

print("\nUnderstat 2nd part\n")
df = x_maker(UTxGA1920[UTxGA1920.columns[30:]],  hist_num, UTxG1920[UTxGA1920.columns[30:]],\
UTodc1920[UTxGA1920.columns[30:]], Team_home1920[TxGA1920.columns[30:]], UTOxG1920[UTxGA1920.columns[30:]].\
mean(axis=1).mean())
X = pd.DataFrame(df[df.columns[:-3]].mean(axis=1))
X['Home'] = df['Home']
X['Opponent'] = df['Opponent']
Y = df['GW']
reg = LinearRegression().fit(X, Y)
display(reg.score(X, Y))
display(reg.coef_)
display(reg.intercept_)
coeff_Understat2 = list(reg.coef_)

print("\nFPL 2020-2021 season\n")
df = x_maker(TxGA,  hist_num, TxG, Todc,Team_home, TOxG.mean(axis=1).mean())
X = pd.DataFrame(df[df.columns[:-3]].mean(axis=1))
X['Home'] = df['Home']
X['Opponent'] = df['Opponent']
Y = df['GW']
reg = LinearRegression().fit(X, Y)
display(reg.score(X, Y))
display(reg.coef_)
display(reg.intercept_)
coeff_FPL3 = list(reg.coef_)

print("\nUnderstat 2020-2021 season\n")
df = x_maker(UTxGA,  hist_num, UTxG, UTodc,Team_home, UTOxG.mean(axis=1).mean())
X = pd.DataFrame(df[df.columns[:-3]].mean(axis=1))
X['Home'] = df['Home']
X['Opponent'] = df['Opponent']
Y = df['GW']
reg = LinearRegression().fit(X, Y)
display(reg.score(X, Y))
display(reg.coef_)
display(reg.intercept_)
coeff_Understat3 = list(reg.coef_)
display(coeff_Understat3)

df_reg = pd.DataFrame({ ' FPL 19-20 1st part': coeff_FPL1, \
' Understat 19-20 1st part': coeff_Understat1, \
' FPL 19-20 2nd part': coeff_FPL2, \
' Understat 19-20 2nd part': coeff_Understat2, \
'FPL 20-21': coeff_FPL3, \
'Understat 20-21': coeff_Understat3}, \
index=['Team xG','Home/Away', 'Opponent defense'])

df_reg = df_reg.style.apply(lambda x: ['Background-color: #09bb9f' if (i==1)&(x[i] >=0) else \
'Background-color: #c4c4c4' if i==1 else '' for i in range(len(x))])

df_reg = df_reg.set_table_styles([{'selector': 'td', 'props': [('border', 'solid 1px')]},\
{'selector': 'th', 'props': [('border', 'solid 1px')]}])

html_reg = df_reg.render()

with open(Path('text/01. H-A/reg.html'), 'w') as file:
    file.write(html_reg)
path_wkhtmltoimage = r'C:\Program Files\wkhtmltopdf\bin\wkhtmltoimage.exe'
config = imgkit.config(wkhtmltoimage=path_wkthmltoimage)
imgkit.from_file('text/01. H-A/reg.html', 'text/01. H-A/reg.jpg', config=config)

df_reg 

FPl All 2019-2020 season

660


0.3058169281961348

array([ 0.60304332, 22.98343527,  1.00442839])

-113.1049977614729

Understat All 2019-2020 season

660


...

array([0.2319734 , 3.15344122, 1.20773186])

-66.18817323744713

[0.23197340052577664, 3.1534412206736357, 1.2077318557346532]

Loading page (1/2)
Rendering (2/2)                                                    
Done                                                               


,FPL 19-20 1st part,Understat 19-20 1st part,FPL 19-20 2nd part,Understat 19-20 2nd part,FPL 20-21,Understat 20-21
Team xG,0.559907,0.545947,0.696984,0.613936,0.449641,0.231973
Home/Away,27.5674,20.4295,-10.7882,-7.4864,4.11424,3.15344
Opponent defense,1.07622,1.15494,1.19235,0.883422,1.10653,1.20773


In [29]:
folder=''
TableFPL = pd.read_csv('in/Table_FPL.csv')
Player_home = pd.read_csv(Path(f'{folder}in/Player_home.csv'))
with open(Path(f'mid/FPL/Player_xG.txt'), 'r') as file:
    Player_xG = pd.DataFrame(json.loads(file.read()))
    Player_xG.index = pd.to_numeric(Player_xG.index)
    PxG = no_lists(Player_xG.sort_index())[no_lists(Player_xG).columns[:7:-1]]

year = '2019-2020'
if year=='': folder = ''
else: folder = f'history/{year}/'
    
Player_home1920 = pd.read_csv(Path(f'{folder}in/Player_home.csv'))
with open(Path(f'{folder}mid/FPL/Player_xG.txt'), 'r') as file:
    Player_xG1920 = pd.DataFrame(json.loads(file.read()))
    Player_xG1920.index = pd.to_numeric(Player_xG1920.index)
    PxG1920 = no_lists(Player_xG1920.sort_index())[no_lists(Player_xG1920).columns[:7:-1]]
    
with open(Path(f'{folder}in/Player_fixtures.txt'), 'r') as file:
    Player_fixtures1920 = pd.DataFrame(json.loads(file.read()))
    Player_fixtures1920.index = pd.to_numeric(Player_fixtures1920.index)
    Pf1920 = no_lists(Player_fixtures1920.sort_index())[PxG1920.columns[::-1]]
# display((PxG1920*Player_home1920).sum(axis=1) - \
# (PxG1920*(Player_home1920-1)).sum(axis=1))
Player_xG1920 = Player_xG1920.sort_index()
Player_fixtures1920 = Player_fixtures1920.sort_index()


df_PHA = pd.DataFrame()
df_PHA['Name'] = Player_xG1920.sort_index()['Name']

df_PHA['xG'] = (PxG1920*Player_home1920).sum(axis=1) - \
(PxG1920*(Player_home1920-1)).sum(axis=1)

df_PHA['H per 90']  = (PxG1920*Player_home1920).sum(axis=1)/(Player_home1920*no_lists(Player_minutes)).sum(axis=1)*90
df_PHA['A per 90']  = (PxG1920*(Player_home1920-1)).sum(axis=1)/((Player_home1920-1)*no_lists(Player_minutes)).sum(axis=1)*90
df_PHA['H-A per 90'] = df_PHA['H per 90'] - df_PHA['A per 90']
df_PHA['H absolute'] = (PxG1920*Player_home1920).sum(axis=1)
df_PHA['A absolute'] = -(PxG1920*(Player_home1920-1)).sum(axis=1)
df_PHA['H-A absolute'] = (PxG1920*Player_home1920).sum(axis=1) + \
(PxG1920*(Player_home1920-1)).sum(axis=1)

#(Player_home1920*no_lists(Player_minutes)).sum(axis=1)/90

df_PHA.sort_values('H-A per 90', ascending = False, inplace = True)
df_PHA_Hmonsters = df_PHA[(df_PHA['H absolute']>200)&(df_PHA['A absolute']>200)].dropna().reset_index().head(10)
del df_PHA_Hmonsters['index']

df_PHA_Hmonsters = df_PHA_Hmonsters.style.set_table_styles([{'selector': 'td', 'props': [('border', 'solid 1px')]},\
{'selector': 'th', 'props': [('border', 'solid 1px')]}])

html_players = df_PHA_Hmonsters.render()

with open(Path('text/01. H-A/playersH.html'), 'w') as file:
    file.write(html_players)
path_wkhtmltoimage = r'C:\Program Files\wkhtmltopdf\bin\wkhtmltoimage.exe'
config = imgkit.config(wkhtmltoimage=path_wkthmltoimage)
imgkit.from_file('text/01. H-A/playersH.html', 'text/01. H-A/playersH.jpg', config=config)


df_PHA.sort_values('H-A per 90', ascending = True, inplace = True)
df_PHA_Amonsters = df_PHA[(df_PHA['H absolute']>200)&(df_PHA['A absolute']>200)].dropna().reset_index().head(10)
del df_PHA_Amonsters['index']

df_PHA_Amonsters = df_PHA_Amonsters.style.set_table_styles([{'selector': 'td', 'props': [('border', 'solid 1px')]},\
{'selector': 'th', 'props': [('border', 'solid 1px')]}])

html_players = df_PHA_Amonsters.render()

with open(Path('text/01. H-A/playersA.html'), 'w') as file:
    file.write(html_players)
path_wkhtmltoimage = r'C:\Program Files\wkhtmltopdf\bin\wkhtmltoimage.exe'
config = imgkit.config(wkhtmltoimage=path_wkthmltoimage)
imgkit.from_file('text/01. H-A/playersA.html', 'text/01. H-A/playersA.jpg', config=config)

df_PHA[(df_PHA['H absolute']>200)&(df_PHA['A absolute']>200)].dropna().reset_index().head(10)
df_PHA_Hmonsters

Loading page (1/2)
Rendering (2/2)                                                    
Done                                                               
Loading page (1/2)
Rendering (2/2)                                                    
Done                                                               


,Name,xG,H per 90,A per 90,H-A per 90,H absolute,A absolute,H-A absolute
0,Riyad Mahrez,1094,63.5463,38.9849,24.5614,663,431,232
1,Jay Rodriguez,827,47.8247,28.0519,19.7727,491,336,155
2,Oliver McBurnie,830,44.3728,24.8276,19.5452,566,264,302
3,Gerard Deulofeu,896,46.7864,29.9711,16.8153,550,346,204
4,Bernardo Mota Veiga de Carvalho e Silva,927,49.7516,33.5261,16.2255,534,393,141
5,Tammy Abraham,1301,62.3004,46.0766,16.2239,659,642,17
6,Anthony Martial,1297,52.1801,37.2848,14.8953,734,563,171
7,David Silva,937,52.0287,37.2984,14.7303,644,293,351
8,Harry Wilson,667,43.99,29.6407,14.3493,392,275,117
9,Todd Cantwell,656,31.4477,17.4725,13.9752,391,265,126


In [27]:
# Players = FPL.Players
# lastGW = FPL.lastGW
# Player_played_fixtures = FPL.Player_played_fixtures
# Table = FPL.Table

year = '2019-2020'
if year=='': folder = ''
else: folder = f'history/{year}/'

with open(Path(f'{folder}in/Player_fixtures.txt'), 'r') as file:
    Player_fixtures1920 = pd.DataFrame(json.loads(file.read()))
    Player_fixtures1920.index = pd.to_numeric(Player_fixtures1920.index)
    Player_fixtures1920 = Player_fixtures1920.sort_index()
    Pf1920 = no_lists(Player_fixtures1920.sort_index())[PxG1920.columns[::-1]]

TableFPL = pd.read_csv(f'{folder}in/Table_FPL.csv')
TableUnderstat = pd.read_csv(f'{folder}in/Table_Understat.csv')
Fixtures = pd.read_csv(Path(f'{folder}in/Fixtures.csv'))
Player_minutes = pd.DataFrame()

for j in range(int(Fixtures['event'].max()),0,-1):
    Player_minutes['GW'+str(j)] = [[] for _ in range(len(PxG1920))]
    for i in range(len(PxG1920)):
        for k in range(len(Player_fixtures1920.at[i, 'GW'+str(j)])):
            Player_minutes.at[i,'GW'+str(j)].append(TableFPL[(TableFPL['fixture']==\
                Player_fixtures1920.at[i, 'GW'+str(j)][k])&(TableFPL['element']==\
                Player_xG1920.at[i,'id'])]['minutes'].sum())
#Player_home = no_lists(Player_home[Player_home.columns[6:]])
Player_minutes = Player_minutes[Player_minutes.columns[::-1]]
#Player_home.to_csv(Path(f'{folder}in/Player_home.csv'), index=False)
no_lists(Player_minutes)
#Player_fixtures1920

,GW1,GW2,GW3,GW4,GW5,GW6,GW7,GW8,GW9,GW10,...,GW39*,GW39,GW40,GW41,GW42,GW43,GW44,GW45,GW46,GW47
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,90.0,90.0,90.0,45.0,90.0,90.0,90.0,0.0,0.0,0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,90.0,90.0,90.0,76.0,14.0,90.0,83.0,0.0,0.0,0
2,0.0,19.0,0.0,90.0,90.0,90.0,90.0,90.0,90.0,15.0,...,90.0,0.0,20.0,90.0,90.0,90.0,83.0,5.0,90.0,33
3,90.0,90.0,90.0,90.0,90.0,40.0,0.0,0.0,0.0,0.0,...,0.0,23.0,10.0,0.0,34.0,1.0,0.0,14.0,0.0,90
4,90.0,90.0,90.0,90.0,90.0,90.0,90.0,90.0,90.0,90.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
5,90.0,90.0,90.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,90.0,0.0,90.0,45.0,0.0,0.0,0.0,90.0,90.0,90
